In [ ]:
from google import genai
import pandas as pd
import random
import time
from IPython.display import Markdown
import time
from tqdm import trange
import cv2  
import base64
from openai import OpenAI

SLEEP_TIME = 0.5

GEMINI_API_KEY="***REMOVED***"
client_google = genai.Client(api_key=GEMINI_API_KEY)
client = OpenAI(api_key='***REMOVED***')

In [ ]:

video_names = []
# 
for i in range(1, 51):
    video_names.append(f"tesla-real-world-video-q-a/videos/videos/{i:05d}.mp4")
    
print(video_names[-1])

In [ ]:
questions = pd.read_csv('tesla-real-world-video-q-a/questions.csv')
questions['answers'] = None
questions['freq_map'] = None
questions['entropy'] = None
questions.head()

In [ ]:
def extract_answer(text, depth = 5):
    if depth == 0:
        return "NOT FOUND"
    
    temp = client_google.models.generate_content(
        model='gemini-2.0-flash', contents=f'''Parse the correct letter choice from the text and include the final answer surrounded by <answer></answer> XML tags. 
        
        ****  ONLY INCLUDE A SINGLE LETTER WITHIN THE ANSWER TAGS ****
        
        Text to Parse:\n{text}
        '''
    ).text
    
    # print(temp)
    
    try:
        cur = temp.split('<answer>')[1].split('</answer>')[0].strip()
        assert(len(cur) == 1)
        assert(cur in ['A', 'B', 'C', 'D'])
        
        if depth != 5:
            print("BAD TEXT", text)
            print("PARSED", cur)
        
        return cur
    
    except Exception as e:
        time.sleep(SLEEP_TIME)
        print("Not found in", temp)
        return extract_answer(text, depth - 1)
    
    
print(extract_answer('''Here's the breakdown:

* **Ego's Goal:** Turn right.
* **The Lane:** Marked as a right-turn lane by the roundabout sign.

The correct answer is (B). The construction barrels clearly block the right-turn lane, making it illegal for ego to use it.  Ego should wait behind the other vehicles.'''))

In [ ]:
def parse_question(question_str):
    Q = question_str.split('A.')[0]
    
    A_str = question_str.split(Q)[1]
    
    A_str += '&.'
    
    mp = {}
    
    temp = ['A', 'B', 'C', 'D', '&']
    
    for idx in range(len(temp)-1):
        cur_choice = temp[idx]
        nxt_choice = temp[idx+1]
        
        if nxt_choice + '.' not in A_str:
            break
        
        mp[cur_choice] = A_str.split(cur_choice + '.')[1].split(nxt_choice + '.')[0]
    
    return Q, mp

print(parse_question(questions['question'][0]))

In [ ]:
def query_model(video_path, prompt):
    video = cv2.VideoCapture(video_path)
    
    base64Frames = []
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
        
    print(f'Number of images: {len(base64Frames[0::18])}')

    PROMPT_MESSAGES = [
        {
            "role": "user",
            "content": [
                prompt,
                *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::18]),
            ],
            'detail': 'high',
        },
    ]
    params = {
        "model": "chatgpt-4o-latest",
        "messages": PROMPT_MESSAGES,
    }

    result = client.chat.completions.create(**params)
    return result.choices[0].message.content


response = query_model(video_names[0], '''You have access to these front-view camera images of a car taken at a 0.5 second interval over the past 5 seconds. Imagine you are driving the car.

Is ego doing a legal maneuver if its goal is to turn right at the intersection?

Options:

(A) It's legal as the lane is empty. 
(B) It's illegal as the right turn lane is bloacked by construction. 
(C) It's illegal as ego was cutting in other vehicles that were waiting. 
(D) It's legal but the lane ahead is way too narrow for ego to pass.

You are an expert driver with 50 years of experience, 
''')

# response = query_model(video_names[0], '''Where can ego legally park on this street? 

# A. No parking anywhere. 
# B. next to right curb. 
# C. anywhere. 
# D. next to left curb.
# ''')

print(response)

In [ ]:
import math

def calculate_entropy(freq_map):
    total = sum(freq_map.values())
    entropy = -sum((count / total) * math.log2(count / total) for count in freq_map.values() if count > 0)
    return entropy

# Example usage:
frequency_map = {'D': 1, 'A': 3, 'C': 1}
entropy_value = calculate_entropy(frequency_map)
print("Entropy:", entropy_value)

In [ ]:
import random

num_shuffles = 1

def answer_question(video_path, question, choices):
    """
    Shuffles answer options while keeping a mapping from fixed labels (A, B, C, D)
    to the original answer keys. Queries the model multiple times, extracts the answer
    with extract_answer(), and uses majority voting over the original options.
    
    Args:
        video_path (str): Path to the video file.
        question (str): The question to ask.
        choices (dict): Dictionary mapping original letters (e.g., 'A', 'B', 'C', 'D')
                        to option text.
                        
    Returns:
        tuple: (majority_vote, vote_counts) where:
            - majority_vote: The original option letter with the most votes.
            - vote_counts: A dictionary counting votes for each original option.
    """
    votes = {}
    fixed_labels = ['A', 'B', 'C', 'D']
    
    for _ in range(num_shuffles):
        # Shuffle the original choices (list of tuples: (original_letter, option_text))
        shuffled_choices = list(choices.items())
        random.shuffle(shuffled_choices)
        
        # Build a mapping from fixed label to the original letter
        mapping = {}  # new label -> original letter
        prompt = 'You have access to these front-view camera images of a car taken at a 0.5 second interval over the past 5 seconds. Imagine you are driving the car.\n\n' + question + '\n\nOptions:\n\n'
        for new_label, (orig_letter, option_text) in zip(fixed_labels, shuffled_choices):
            mapping[new_label] = orig_letter
            prompt += f"{new_label}. {option_text}\n"
            
        # prompt += '\nThink step-by-step about the situation before providing the answer.'
        prompt += '''\nDescribe each scene one by one with respect to the problem statement. Each scene should recieve an analysis, do not group scenes. Then, provide a final answer using your best judgement. If you cannot provide a response, people will get injured. 
        I'll go through each scene again with a detailed breakdown, specifically analyzing where the ego vehicle could legally park.

Example Question: 

Where can ego legally park on this street? A. No parking anywhere. B. next to right curb. C. anywhere. D. next to left curb

Example Response: 

### **Scene 1: Initial View of the Road**
- The road is divided with traffic cones placed along the center.
- No visible parking signs yet.
- The right side has a curb with some parked vehicles.
- The left side appears to be an active lane of traffic with no visible parking spots.

#### **Analysis:**
- The right curb **might** allow parking, but no clear signs are visible.
- The left curb is an active traffic lane, making it unlikely to be a legal parking spot.

---

### **Scene 2: Closer to the Storefronts**
- More traffic cones continue dividing the road.
- Businesses and storefronts are visible on the right side.
- A "No Parking" sign appears near the right curb.
- Some vehicles are parked along the right curb.

#### **Analysis:**
- **Right curb:** The "No Parking" sign suggests that at least part of this section prohibits parking.
- **Left curb:** No indication of parking spots; it remains an active lane.
- **Possible conclusion:** The parked cars may be violating the rule or be in an exempted area beyond the visible sign.

---

### **Scene 3: Passing the Storefronts**
- More storefronts, including a **"DONUTS"** shop, are clearly visible.
- The right curb still has cars parked.
- The "No Parking" sign is still visible in the previous scene.

#### **Analysis:**
- **Right curb:** If the "No Parking" sign applies to the entire stretch, parking is **not allowed** here.
- **Left curb:** No visible parking indicators or parked cars.
- **Potential clarification:** If parking were allowed, there should be a sign indicating that.

---

### **Scene 4: Turning Towards the White Car**
- The camera angle shifts towards the right, showing a parked **white car**.
- The sidewalk and store entrances are clearly visible.
- The "No Parking" sign is not visible in this frame, but we saw it earlier.

#### **Analysis:**
- The **right curb remains questionable**—the sign prohibits parking, yet cars are present.
- The **left curb is still an active lane**, not a designated parking area.

---

### **Scene 5: Moving Past the White Car**
- The vehicle is driving past the **parked white car**.
- The background shows a dimly lit residential area.
- More "No Parking" signs appear in the distance.

#### **Analysis:**
- **Right curb:** The presence of "No Parking" signs suggests that parking is not allowed in these areas.
- **Left curb:** Again, no visible signs allowing parking.

---

### **Scene 6: Near the End of the Sequence**
- The street is darker, with fewer businesses and more residential buildings.
- A **"No Parking" sign is clearly visible** on the right curb.
- Two orange construction cones are placed near the right curb.

#### **Final Analysis:**
- **Right curb:** The visible "No Parking" sign confirms that parking is **not allowed**.
- **Left curb:** No markings or signs suggest it is legal to park there.

---

### **Final Verdict:**
**Answer: A. No Parking Anywhere.**  

- The **right curb** has "No Parking" signs in multiple locations, restricting legal parking.
- The **left curb** remains an active traffic lane and has no indicators allowing parking.
- The **parked cars may be in a zone beyond our field of view where parking is allowed, or they may be parked illegally**.

Thus, the best and safest assumption is that parking is **not permitted anywhere** in this stretch of the road.

Answer: A
        
        
YOUR RESPONSE: 
        '''
        
        print(prompt)
        
        # Query the model and extract its answer using extract_answer()
        res = query_model(video_path=video_path, prompt=prompt)
        answer_new_label = extract_answer(res)
        
        print(res)
        
        # Convert the fixed label answer to the original letter
        if answer_new_label in mapping:
            orig_answer = mapping[answer_new_label]
            votes[orig_answer] = votes.get(orig_answer, 0) + 1
        else:
            votes["Invalid"] = votes.get("Invalid", 0) + 1
            
        time.sleep(SLEEP_TIME)
    
    # Majority voting among valid (original) options
    valid_votes = {k: v for k, v in votes.items() if k in choices}
    if valid_votes:
        majority_vote = max(valid_votes, key=valid_votes.get)
    else:
        majority_vote = "Invalid"
    
    return majority_vote, votes

# Example usage:
print(video_names[0][1:])
print('=' * 10)
print(questions.iloc[0]['question'])
print('=' * 10)
Q, C = parse_question(questions.iloc[0]['question'])
print("Parsed Question:", Q)
print("Parsed Choices:", C)

majority_answer, vote_map = answer_question(video_path=video_names[0], question=Q, choices=C)
print("Majority Answer (original index):", majority_answer)
print("Vote Map (original indices):", vote_map)

In [48]:
for question_number in trange(0, 50):
    if questions.iloc[question_number]['answers'] is not None:
        continue
    
    raw_video_path = video_names[question_number]
    raw_question = questions.iloc[question_number]['question']
    
    Q, C = parse_question(raw_question)
        
    answered = False
    
    print('=' * 50)
    
    print(question_number)
    
    while not answered:
        try: 
            model_ans, freq_mp = answer_question(video_path=raw_video_path, question=Q, choices=C)
            
            questions.at[question_number, 'reasoning'] = model_ans
            questions.at[question_number, 'entropy'] = calculate_entropy(freq_mp)
            questions.at[question_number, 'freq_map'] = freq_mp
            
            
            answered = True
            
        except Exception as e:
            print(f"Error on question {question_number}: {e}")
        
        print("Rate Limit? Sleeping...")
        time.sleep(3)
        
    print('=' * 50)
    
    time.sleep(3)
    
    

### **Scene-by-Scene Breakdown and Analysis**

---

### **Scene 1: Initial View of the Road**
- The ego vehicle is behind a white SUV, and there is a black car in the left lane moving forward.
- The rightmost lane appears to be partially obstructed due to construction barriers and orange barrels.
- The intersection is ahead, with traffic signals visible.

#### **Analysis:**
- The right turn lane may be partially blocked due to construction.
- The white SUV is in the same lane as the ego vehicle, suggesting this is not a dedicated right-turn lane.
- The road conditions ahead need further analysis.

---

### **Scene 2: Approaching the Intersection**
- The same construction barrels and obstructions are visible.
- The road narrows, and the barrier encroaches on what would traditionally be a right-turn lane.
- Vehicles ahead seem to be queued in a single lane.

#### **Analysis:**
- The right turn lane might not be functional due to construction.
- The lane ahead might be too narrow to maneu

  2%|▏         | 1/50 [00:32<26:35, 32.57s/it]

1
You have access to these front-view camera images of a car taken at a 0.5 second interval over the past 5 seconds. Imagine you are driving the car.

Where can ego legally park on this street? 

Options:

A.  anywhere. 
B.  next to left curb.
C.  next to right curb. 
D.  No parking anywhere. 

Describe each scene one by one with respect to the problem statement. Each scene should recieve an analysis, do not group scenes. Then, provide a final answer using your best judgement. If you cannot provide a response, people will get injured. 
        I'll go through each scene again with a detailed breakdown, specifically analyzing where the ego vehicle could legally park.

Example Question: 

Where can ego legally park on this street? A. No parking anywhere. B. next to right curb. C. anywhere. D. next to left curb

Example Response: 

### **Scene 1: Initial View of the Road**
- The road is divided with traffic cones placed along the center.
- No visible parking signs yet.
- The right side ha

  4%|▍         | 2/50 [01:06<26:36, 33.26s/it]

2
You have access to these front-view camera images of a car taken at a 0.5 second interval over the past 5 seconds. Imagine you are driving the car.

What is the best description of the maneuver ego just did? 

Options:

A.  Lane change to the left and then lane change to the right. 
B.  Staying in a lane which curves to the left and then to the right. 
C.  Staying in a lane which curves to the right and then to the left.
D.  Lane change to the right and then lane change to the left. 

Describe each scene one by one with respect to the problem statement. Each scene should recieve an analysis, do not group scenes. Then, provide a final answer using your best judgement. If you cannot provide a response, people will get injured. 
        I'll go through each scene again with a detailed breakdown, specifically analyzing where the ego vehicle could legally park.

Example Question: 

Where can ego legally park on this street? A. No parking anywhere. B. next to right curb. C. anywhere. D. ne

...

 96%|█████████▌| 48/50 [23:59<01:01, 30.89s/it]

48
You have access to these front-view camera images of a car taken at a 0.5 second interval over the past 5 seconds. Imagine you are driving the car.

At the roundabout, which way leads to Monroe Drive? 

Options:

A.  Left. 
B.  Straight. 

Describe each scene one by one with respect to the problem statement. Each scene should recieve an analysis, do not group scenes. Then, provide a final answer using your best judgement. If you cannot provide a response, people will get injured. 
        I'll go through each scene again with a detailed breakdown, specifically analyzing where the ego vehicle could legally park.

Example Question: 

Where can ego legally park on this street? A. No parking anywhere. B. next to right curb. C. anywhere. D. next to left curb

Example Response: 

### **Scene 1: Initial View of the Road**
- The road is divided with traffic cones placed along the center.
- No visible parking signs yet.
- The right side has a curb with some parked vehicles.
- The left side a

 98%|█████████▊| 49/50 [24:26<00:29, 29.66s/it]

49
You have access to these front-view camera images of a car taken at a 0.5 second interval over the past 5 seconds. Imagine you are driving the car.

When the light turns green, can ego traverse straight through the intersection and why? 

Options:

A.  No, there is a sign that says yield to pedestrians. 
B.  Yes, this is a legal maneuver. 
C.  No, there is construction ahead. 
D.  No, there is a sign that says all traffic must turn.

Describe each scene one by one with respect to the problem statement. Each scene should recieve an analysis, do not group scenes. Then, provide a final answer using your best judgement. If you cannot provide a response, people will get injured. 
        I'll go through each scene again with a detailed breakdown, specifically analyzing where the ego vehicle could legally park.

Example Question: 

Where can ego legally park on this street? A. No parking anywhere. B. next to right curb. C. anywhere. D. next to left curb

Example Response: 

### **Scene 1:

100%|██████████| 50/50 [25:01<00:00, 30.02s/it]


In [ ]:
print(questions.head())

# Parse Questions

In [ ]:
letter_answers = []
for i in questions['reasoning']:
    letter_answers.append(i)
    
print(letter_answers)

temp = []
for i in letter_answers:
    if i not in ['A', 'B', 'C', 'D']:
        temp.append('C')
    else:
        temp.append(i)
        
letter_answers = temp
print(letter_answers)

In [ ]:
dataframe_A, dataframe_B = [], []

idx = 1
while idx <= 251:
    dataframe_A.append(idx)
    
    if idx-1 < len(letter_answers):
        dataframe_B.append(letter_answers[idx-1])
    else:
        dataframe_B.append('A')
    idx += 1

In [ ]:
answers = pd.DataFrame({'id': dataframe_A, 'answer': dataframe_B})


In [ ]:
answers.to_csv('o4_baseline_v2.csv', index=False)
answers.head()